# Radex

In [21]:
from spectralradex import radex
from multiprocessing import Pool
import numpy as np
import time


The simplest use case for SpectralRadex is to be a simple python wrapper for RADEX. This allows large grids of RADEX models or complex parameter inference procedures to be run in an environment suited to those tasks.

If one wishes to run radex, we simply need a dictionary of the parameters RADEX expects. An example can be obtained using the ```get_default_parameters()``` function like so

In [9]:
params = radex.get_default_parameters()
print("{")
for key,value in params.items():
    print(f"\t{key} : {value}")
print("}")

{
	molfile : co.dat
	tkin : 30.0
	tbg : 2.73
	cdmol : 10000000000000.0
	h2 : 100000.0
	h : 0.0
	e- : 0.0
	p-h2 : 0.0
	o-h2 : 0.0
	h+ : 0.0
	linewidth : 1.0
	fmin : 0.0
	fmax : 30000000.0
}


and then we pass that to the ```run()``` function.

In [11]:
output = radex.run(params)
output.head()

,E_UP (K),freq,WAVEL (um),T_ex,tau,T_R (K),POP UP,POP LOW,FLUX (K*km/s),FLUX (erg/cm2/s),Qup,Qlow
0,5.53,115.271202,2600.757633,31.666252,0.000223,0.006275,0.246666,0.097917,0.006680,1.317591e-10,1,0
1,16.60,230.538000,1300.403656,29.262261,0.000735,0.017551,0.281677,0.246666,0.018683,2.947981e-09,2,1
2,33.19,345.795990,866.963374,26.640080,0.001112,0.021294,0.211510,0.281677,0.022667,1.207049e-08,3,2
3,55.32,461.040768,650.251515,24.363876,0.001022,0.015261,0.109663,0.211510,0.016246,2.050309e-08,4,3
4,82.97,576.267931,520.231028,22.798547,0.000605,0.007078,0.039845,0.109663,0.007535,1.856956e-08,5,4


## Parameter Grids
It is more likely that one will want to run the code over many combinations of input parameters. This can be achieved via the ```run_grid()``` function. This function takes iterables for the three variables (density, temperature and column density) as well as fixed values for the other RADEX parameters. It then produces the RADEX output for all combinations of the three iterables.

In [22]:
tic = time.perf_counter()

grid_df = radex.run_grid(density_values=np.arange(1.0e5, 1.0e6, 1.0e5), temperature_values=np.arange(10, 100, 10),
                   column_density_values=np.arange(1.0e14, 1.0e15, 1.0e14), molfile='co.dat',
                   target_value="T_R (K)")
toc = time.perf_counter()
print(f"run_grid took {toc-tic:0.4f} seconds without a pool")

run_grid took 12.6135 seconds without a pool


In [24]:
grid_df.iloc[:,0:6].head()

,Density,Temperature,Column Density,(1)-(0)[115.2712018 GHz],(2)-(1)[230.538 GHz],(3)-(2)[345.7959899 GHz]
0,100000.0,10.0,1.000000e+14,0.102858,0.140506,0.054774
1,100000.0,20.0,1.000000e+14,0.079719,0.185064,0.166824
2,100000.0,30.0,1.000000e+14,0.062672,0.174883,0.211890
3,100000.0,40.0,1.000000e+14,0.051761,0.158660,0.224313
4,100000.0,50.0,1.000000e+14,0.044285,0.143712,0.223537


### Parallelization
In order to be as flexible as possible, SpectralRadex has no built in multiprocessing. However, the ```run_grid()``` function does take the optional parameter ```pool``` which should be an object with ```map()```, ```join()```, and ```close()``` methods that allow functions to be evaluated in parallel. For example, the python standard [multiprocessing.pool](https://docs.python.org/3.6/library/multiprocessing.html) obect or Schwimmbad's [MPIPool](https://schwimmbad.readthedocs.io/en/latest/examples/#using-mpipool).

If such an object is supplied, the grid will be evaluated in parallel. Note the time in the example below compared to the grid above.

In [23]:
tic = time.perf_counter()
pool=Pool(8)
grid_df = radex.run_grid(density_values=np.arange(1.0e5, 1.0e6, 1.0e5), temperature_values=np.arange(10, 100, 10),
                   column_density_values=np.arange(1.0e14, 1.0e15, 1.0e14), molfile='co.dat',
                   target_value="T_R (K)",pool=pool)
toc = time.perf_counter()
print(f"run_grid took {toc-tic:0.4f} seconds with a pool of 8 workers")

(728, 43)
run_grid took 2.2611 seconds with a pool of 8 workers
